In [ ]:
#! pip install trubrics

## Load titanic data

In [ ]:
from trubrics.example import get_titanic_data_and_model
train_df, test_df, model = get_titanic_data_and_model()

## Init DataContext

In [ ]:
from trubrics.context import DataContext

In [ ]:
data_context = DataContext(
    name="my_first_dataset",
    version=0.1,
    training_data=train_df,
    testing_data=test_df,
    target="Survived"
)

## Custom model / algorithm

In [ ]:
class ExampleCustomModel:
    """
    This is a custom model that scores passengers based on probability of survival.
    """
    def __init__(self, model):
        self.model = model
        self._estimator_type = "regressor"

    def predict(self, df):
        return self.model.predict(df)
    
    def fit(self, X_train, y_train):
        return self.model.fit(X_train, y_train)
    
    def get_params(self, deep):
        return {"model": self.model}

In [ ]:
# example first 5 scores
ExampleCustomModel(model=model).predict(data_context.X_test)[:5]

In [ ]:
custom_model = ExampleCustomModel(model=model)

## Build custom validations

Lets build the following custom validations:

**Data validations**
- validate that passengers with the title "master" are younger than a certain age

**Training validations**
- validate that CV scores are within a certain std

**Model validations**
- validate that the model scores are on average higher for females than for males

In [ ]:
from examples.classification_titanic.training_validations.custom_validator import CustomValidator

In [ ]:
validator = CustomValidator(data=data_context, model=custom_model)

In [ ]:
validations = [
    validator.validate_master_age(age_limit_master=13),
    validator.validate_model_scores_females_higher(severity="experiment"),
    validator.validate_cv_scores(cv=5, threshold_std=0.05, metric="precision"),
    validator.validate_cv_scores(cv=5, threshold_std=0.05, metric="recall"),
]

## Save validations as a trubric

In [ ]:
from trubrics.validations import Trubric

trubric = Trubric(
    trubric_name="my_first_trubric",
    model_name="my_model",
    model_version=0.1,
    data_context_name=data_context.name,
    data_context_version=data_context.version,
    metadata={"tag": "master"}, # tag any metadata here
    validations=validations,
)

In [ ]:
# save trubric to a local .json
trubric.save_local(path=".")

## Run trubric from CICDCT

The last step is to run the validations from the trubric .json against new data / models everytime there is any potential change. See our [CLI docs](https://trubrics.github.io/trubrics-sdk/trubrics_cli/) for info.